# Phase 3: 서비스 레이어 - API 연동

## 🎯 이 노트북에서 배울 것
- **API**가 무엇이고 왜 필요한지 (식당 주문 비유)
- **HTTP 요청/응답**의 기본 개념
- **예외 처리(try-except)**가 왜 필요하고 어떻게 사용하는지
- 실제 API(Tavily, Gemini) 연동 코드 이해하기
- 서비스 레이어의 역할 이해하기

## 📋 사전 준비
- Phase 1, 2 완료 (Settings, NewsArticle 클래스)
- `.env` 파일에 API 키 설정 (TAVILY_API_KEY, GEMINI_API_KEY)
- 필요한 패키지 설치: `tavily-python`, `google-genai`

## 📖 학습 순서
1. **이론 학습** - 개념을 이해하고 실행하며 확인
2. **가이드 실습** - TODO를 따라 직접 코딩 (답 제공)
3. **프로젝트 코드 분석** - 실제 프로젝트에서 어떻게 사용되는지
4. **강사와 함께하는 챌린지** - 스스로 문제 해결 (답 없음)

---

# 📚 이론 학습: 실행하며 배우기

---

## 📚 API란 무엇인가요?

### 정의

**API(Application Programming Interface)**는 프로그램들이 서로 통신하는 방법을 정의한 인터페이스입니다.

| 용어 | 설명 |
|------|------|
| API | 프로그램 간 통신 규약 |
| 요청(Request) | API에 보내는 데이터 |
| 응답(Response) | API가 돌려주는 데이터 |
| API 키 | 인증 정보 (누가 요청하는지) |

### 왜 필요한가요?

우리 프로그램에서 직접 구현하기 어려운 기능을 **다른 서비스에서 가져다 쓸 수 있습니다!**

- 뉴스 검색? → **Tavily API**가 해줍니다
- AI 요약? → **Gemini API**가 해줍니다
- 날씨 정보? → 기상청 API가 해줍니다
- 지도? → Google Maps API가 해줍니다

### 실행해서 확인해봅시다!
API의 개념을 코드로 이해해봅시다.

In [1]:
# ============================================
# 🔍 API 개념 이해하기 - 식당 비유
# ============================================
# API를 식당에 비유하여 이해해봅시다.

# 식당 시뮬레이션 (비유를 위한 코드)
class 식당API:
    """API는 이렇게 동작합니다!"""
    
    def __init__(self):
        # 메뉴판 (API가 제공하는 기능들)
        self.메뉴 = {
            "짜장면": 7000,
            "짬뽕": 8000,
            "탕수육": 15000
        }
    
    def 주문(self, 메뉴이름: str) -> dict:
        """API 요청을 받아 응답을 반환"""
        # 요청 처리
        if 메뉴이름 in self.메뉴:
            # 성공 응답
            return {
                "status": "성공",
                "음식": 메뉴이름,
                "가격": self.메뉴[메뉴이름]
            }
        else:
            # 실패 응답 (메뉴에 없음)
            return {
                "status": "실패",
                "에러": "메뉴에 없는 음식입니다"
            }

# API 사용하기
식당 = 식당API()

# 요청 1: 성공적인 주문
응답1 = 식당.주문("짜장면")
print("=== 주문 1: 짜장면 ===")
print(f"응답: {응답1}")

# 요청 2: 실패한 주문
응답2 = 식당.주문("피자")
print("\n=== 주문 2: 피자 ===")
print(f"응답: {응답2}")

# 💡 API도 이와 같습니다!
# - 요청(Request)을 보내면
# - 응답(Response)이 돌아옵니다
# - 성공하면 데이터가, 실패하면 에러 정보가 옵니다

=== 주문 1: 짜장면 ===
응답: {'status': '성공', '음식': '짜장면', '가격': 7000}

=== 주문 2: 피자 ===
응답: {'status': '실패', '에러': '메뉴에 없는 음식입니다'}


### 📝 핵심 정리

| 용어 | 설명 |
|------|------|
| **API** | 프로그램 간 통신 인터페이스 |
| **요청(Request)** | API에 보내는 데이터 |
| **응답(Response)** | API가 돌려주는 데이터 |
| **API 키** | 인증 정보 |

💡 **기억하세요**: API는 "다른 서비스의 기능을 빌려 쓰는 방법"입니다!

---

## 📚 HTTP 요청/응답 이해하기

### 이게 뭔가요?

HTTP는 **인터넷에서 대화하는 언어**입니다! 🌐

웹 브라우저에서 사이트를 열 때, 서버에 API 요청을 보낼 때 모두 HTTP를 사용합니다.

```
[요청 예시]
GET https://api.example.com/news?keyword=AI
Authorization: Bearer sk-xxxxx  ← API 키

[응답 예시]
HTTP 200 OK  ← 상태 코드 (성공)
{
  "results": [{"title": "AI 뉴스", ...}]
}
```

### 주요 HTTP 상태 코드

| 코드 | 의미 | 설명 |
|------|------|------|
| **200** | OK | 성공! ✅ |
| **400** | Bad Request | 요청이 잘못됨 (오타 등) |
| **401** | Unauthorized | 인증 실패 (API 키 오류) |
| **429** | Too Many Requests | 요청이 너무 많음 (Rate Limit) |
| **500** | Server Error | 서버 문제 |

### 실행해서 확인해봅시다!
실제 HTTP 요청을 보내봅시다.

In [9]:
# ============================================
# 🔍 HTTP 요청 보내기 (requests 라이브러리)
# ============================================
# 실제 API에 HTTP 요청을 보내봅시다.

import requests  # <- HTTP 요청을 보내는 라이브러리
from pprint import pprint

# 공개 API에 요청 보내기 (httpbin.org는 테스트용 API)
url = "https://httpbin.org/get"  # GET 요청 테스트 API

print("=== HTTP GET 요청 보내기 ===")
print(f"URL: {url}")

# 요청 보내기
response = requests.get(url)  # GET 요청
pprint(response.json())

# 응답 확인
print(f"\n상태 코드: {response.status_code}")  # 200이면 성공
print(f"성공 여부: {response.ok}")  # True/False

# 응답 데이터 (JSON)
data = response.json()  # JSON을 딕셔너리로 변환
print(f"\n응답 데이터 일부:")
print(f"  - origin (요청자 IP): {data.get('origin', '없음')}")
print(f"  - url: {data.get('url', '없음')}")

# 💡 이것이 HTTP 요청/응답의 기본입니다!

=== HTTP GET 요청 보내기 ===
URL: https://httpbin.org/get
{'args': {},
 'headers': {'Accept': '*/*',
             'Accept-Encoding': 'gzip, deflate',
             'Host': 'httpbin.org',
             'User-Agent': 'python-requests/2.32.5',
             'X-Amzn-Trace-Id': 'Root=1-6980111b-17bcd75c62d847565507b019'},
 'origin': '121.140.83.123',
 'url': 'https://httpbin.org/get'}

상태 코드: 200
성공 여부: True

응답 데이터 일부:
  - origin (요청자 IP): 121.140.83.123
  - url: https://httpbin.org/get


In [ ]:
# ============================================
# 🔍 HTTP 상태 코드 이해하기
# ============================================
# 여러 상태 코드를 테스트해봅시다.

import requests

# 테스트할 URL들 (httpbin은 원하는 상태코드를 반환해줌)
test_cases = [
    ("https://httpbin.org/status/200", "성공"),
    ("https://httpbin.org/status/401", "인증 실패"),
    ("https://httpbin.org/status/429", "요청 과다"),
    ("https://httpbin.org/status/500", "서버 오류"),
]

print("=== HTTP 상태 코드 테스트 ===")
for url, description in test_cases:
    try:
        response = requests.get(url, timeout=5)
        status = response.status_code
        emoji = "✅" if response.ok else "❌"
        print(f"{emoji} {status}: {description}")
    except Exception as e:
        print(f"⚠️ 오류: {e}")

# 💡 상태 코드로 요청이 성공했는지, 왜 실패했는지 알 수 있습니다!

### 📝 핵심 정리

| 항목 | 설명 |
|------|------|
| **HTTP** | 웹에서 데이터를 주고받는 프로토콜 |
| **GET** | 데이터를 가져오는 요청 |
| **POST** | 데이터를 보내는 요청 |
| **상태 코드** | 요청 결과를 나타내는 숫자 (200=성공) |
| **JSON** | 데이터 교환 형식 (딕셔너리와 비슷) |

💡 **기억하세요**: 상태 코드 200이면 성공, 400대/500대면 문제 발생!

---

## 📚 예외 처리(try-except)란?

### 정의

예외 처리는 **프로그램 실행 중 발생할 수 있는 오류를 안전하게 처리하는 방법**입니다.

```python
# 예외 처리 없이 (위험!)
result = 10 / 0  # 프로그램 전체가 죽음! 💀

# 예외 처리 있으면 (안전)
try:
    result = 10 / 0
except ZeroDivisionError:
    print("0으로 나눌 수 없습니다")  # 프로그램 계속 실행! ✅
```

### 왜 필요한가요?

**API 호출은 실패할 수 있습니다:**
- 인터넷이 끊겼을 때
- API 키가 잘못되었을 때
- 요청을 너무 많이 보냈을 때
- 서버에 문제가 있을 때

예외 처리가 없으면 → 앱이 죽음 💀
예외 처리가 있으면 → 사용자에게 친절한 메시지 표시 ✅

### 실행해서 확인해봅시다!

In [10]:
# ============================================
# 🔍 예외 처리 기본 사용법
# ============================================
# try-except의 기본 구조를 알아봅시다.

# 예시 1: 0으로 나누기
print("=== 예시 1: 0으로 나누기 ===")
try:
    result = 10 / 0  # 오류 발생!
    print(f"결과: {result}")  # 이 줄은 실행 안 됨
except ZeroDivisionError:  # 이 오류를 잡음
    print("❌ 0으로 나눌 수 없습니다!")

print("✅ 프로그램이 계속 실행됩니다!\n")

# 예시 2: 존재하지 않는 키 접근
print("=== 예시 2: 딕셔너리 키 오류 ===")
data = {"name": "홍길동"}
try:
    age = data["age"]  # 'age' 키가 없음!
except KeyError:
    print("❌ 'age' 키가 존재하지 않습니다!")

print("✅ 프로그램이 계속 실행됩니다!\n")

# 예시 3: 여러 종류의 예외 처리
print("=== 예시 3: 여러 예외 처리 ===")
try:
    value = int("abc")  # 숫자로 변환 불가!
except ValueError:
    print("❌ 숫자로 변환할 수 없습니다!")
except TypeError:
    print("❌ 타입이 잘못되었습니다!")

# 💡 try 안에서 오류가 나면 except로 점프합니다!

=== 예시 1: 0으로 나누기 ===
❌ 0으로 나눌 수 없습니다!
✅ 프로그램이 계속 실행됩니다!

=== 예시 2: 딕셔너리 키 오류 ===
❌ 'age' 키가 존재하지 않습니다!
✅ 프로그램이 계속 실행됩니다!

=== 예시 3: 여러 예외 처리 ===
❌ 숫자로 변환할 수 없습니다!


In [11]:
# ============================================
# 🔍 예외 처리 심화: finally와 else
# ============================================
# try-except의 확장된 형태를 알아봅시다.

def divide(a, b):
    """나눗셈을 수행하고 결과를 반환"""
    try:
        result = a / b
    except ZeroDivisionError:
        print("  ❌ 0으로 나눌 수 없습니다")
        return None
    else:
        # 오류가 없을 때만 실행
        print(f"  ✅ 성공! 결과: {result}")
        return result
    finally:
        # 성공이든 실패든 항상 실행
        print("  📋 나눗셈 시도 완료")

print("=== 성공 케이스 ===")
divide(10, 2)

print("\n=== 실패 케이스 ===")
divide(10, 0)

# 💡 구조 정리:
# try: 시도할 코드
# except: 오류 발생 시 실행
# else: 오류 없을 때만 실행 (선택)
# finally: 항상 실행 (선택)

=== 성공 케이스 ===
  ✅ 성공! 결과: 5.0
  📋 나눗셈 시도 완료

=== 실패 케이스 ===
  ❌ 0으로 나눌 수 없습니다
  📋 나눗셈 시도 완료


In [13]:
# ============================================
# 🔍 커스텀 예외 만들기
# ============================================
# 우리만의 예외 클래스를 만들어봅시다.

# 커스텀 예외 클래스 정의
class AppError(Exception):  # Exception을 상속
    """애플리케이션 전용 예외"""
    def __init__(self, error_type: str):
        self.error_type = error_type  # 에러 종류 저장
        super().__init__(error_type)  # 부모 클래스 초기화

# 에러 메시지 딕셔너리
ERROR_MESSAGES = {
    "api_key_invalid": "API 키가 유효하지 않습니다.",
    "rate_limit_exceeded": "요청이 너무 많습니다. 잠시 후 다시 시도해주세요.",
    "network_error": "네트워크 연결을 확인해주세요."
}

# 커스텀 예외 사용
def fake_api_call(api_key: str):
    """API 호출 시뮬레이션"""
    if api_key == "":
        raise AppError("api_key_invalid")  # 커스텀 예외 발생!
    return {"data": "성공"}

print("=== 커스텀 예외 테스트 ===")

# 테스트 1: 유효한 키
try:
    result = fake_api_call("valid-key")
    print(f"✅ 성공: {result}")
except AppError as e:
    message = ERROR_MESSAGES.get(e.error_type, "알 수 없는 오류")
    print(f"❌ 실패: {message}")

# 테스트 2: 빈 키
try:
    result = fake_api_call("")  # 빈 키 → 예외 발생
    print(f"✅ 성공: {result}")
except AppError as e:
    message = ERROR_MESSAGES.get(e.error_type, "알 수 없는 오류")
    print(f"❌ 실패: {message}")

# 💡 프로젝트에서는 AppError를 사용하여
# API 오류를 사용자 친화적으로 처리합니다!

=== 커스텀 예외 테스트 ===
✅ 성공: {'data': '성공'}
❌ 실패: API 키가 유효하지 않습니다.


### 📖 참고: 파이썬 공식 예외(Exception) 정리

파이썬에는 미리 정의된 **내장 예외**들이 있습니다.

**자주 사용하는 예외:**

| 예외 | 발생 상황 | 예시 |
|------|----------|------|
| `ValueError` | 값이 잘못됨 | `int("abc")` |
| `TypeError` | 타입이 잘못됨 | `"hello" + 5` |
| `KeyError` | 딕셔너리 키 없음 | `d["없는키"]` |
| `IndexError` | 리스트 인덱스 초과 | `[1,2,3][10]` |
| `AttributeError` | 속성/메서드 없음 | `"str".없는메서드()` |
| `FileNotFoundError` | 파일 없음 | `open("없는파일.txt")` |
| `ZeroDivisionError` | 0으로 나눔 | `10 / 0` |
| `ImportError` | import 실패 | `import 없는모듈` |

**예외 계층 구조:**
```
BaseException
 └── Exception          ← 대부분의 예외는 여기서 상속
      ├── ValueError
      ├── TypeError
      ├── KeyError
      ├── RuntimeError
      └── ... (우리의 AppError도 여기서 상속!)
```

**예외 잡는 순서:**
```python
try:
    # 위험한 코드
except ValueError:      # 1. 구체적인 예외 먼저
    print("값 오류")
except TypeError:       # 2. 다른 구체적인 예외
    print("타입 오류")
except Exception:       # 3. 나머지 모든 예외 (맨 마지막에!)
    print("기타 오류")
```

💡 **팁**: `except Exception:`은 거의 모든 예외를 잡으므로 맨 마지막에 사용하세요!

### 📖 참고: `super().__init__()`이란?

**`super()`는 부모 클래스의 메서드를 호출할 때 사용합니다.**

```python
class Parent:
    def __init__(self, name):
        self.name = name
        print(f"부모 초기화: {name}")

class Child(Parent):
    def __init__(self, name, age):
        super().__init__(name)  # 부모의 __init__ 호출!
        self.age = age          # 자식만의 추가 속성
        print(f"자식 초기화: {age}세")

# 사용
child = Child("철수", 10)
# 출력:
# 부모 초기화: 철수
# 자식 초기화: 10세
```

**왜 필요한가요?**

```python
class AppError(Exception):
    def __init__(self, error_type: str):
        self.error_type = error_type
        super().__init__(error_type)  # ← 이게 왜 필요할까?
```

| 호출 | 역할 |
|------|------|
| `self.error_type = error_type` | 우리가 추가한 속성 저장 |
| `super().__init__(error_type)` | Exception의 기본 기능 초기화 (에러 메시지 설정 등) |

**`super().__init__()`을 안 부르면?**
- 부모 클래스의 초기화가 안 됨
- 예외의 기본 기능(메시지 출력 등)이 제대로 작동 안 할 수 있음

💡 **규칙**: 부모를 상속받으면 `super().__init__(...)`으로 부모도 초기화하세요!

### 📖 참고: 클래스에서 상속(Inheritance) 받기

**상속은 기존 클래스의 기능을 물려받아 새 클래스를 만드는 것입니다.**

```python
# 부모 클래스
class Animal:
    def speak(self):
        return "소리를 냅니다"

# 자식 클래스 (Animal을 상속)
class Dog(Animal):  # <- 괄호 안에 부모 클래스
    def speak(self):  # 부모의 메서드를 재정의(오버라이드)
        return "멍멍!"

class Cat(Animal):
    def speak(self):
        return "야옹!"
```

**상속의 장점:**
| 장점 | 설명 |
|------|------|
| **코드 재사용** | 부모의 기능을 그대로 사용 |
| **확장** | 부모 기능 + 새로운 기능 추가 |
| **일관성** | 같은 부모를 가진 클래스들은 비슷하게 동작 |

**커스텀 예외에서의 상속:**
```python
class AppError(Exception):  # Exception을 상속!
    pass

# AppError는 Exception의 모든 기능을 가짐
# + 우리가 추가한 기능도 가짐
```

💡 `class 자식(부모):` 형태로 상속받습니다!

### 📝 핵심 정리

| 구문 | 역할 |
|------|------|
| `try:` | 오류가 날 수 있는 코드 |
| `except 예외:` | 해당 예외 발생 시 실행 |
| `else:` | 예외 없이 성공 시 실행 |
| `finally:` | 항상 실행 (정리 작업) |
| `raise` | 예외를 발생시킴 |

💡 **기억하세요**: 예외 처리는 프로그램을 안전하게 만드는 안전벨트입니다!

---

## 📚 서비스 레이어란?

### 정의

서비스 레이어는 **외부 API 호출과 비즈니스 로직을 담당하는 계층**입니다.

- 외부(API)와 소통하는 역할
- 비즈니스 로직을 처리
- UI(화면)와 데이터 저장소 사이의 중간 다리

```
[UI] ←→ [서비스 레이어] ←→ [외부 API]
           ↓
      [데이터 저장소]
```

### 왜 필요한가요?

1. **관심사 분리**: UI는 화면만, 서비스는 로직만
2. **재사용**: 같은 서비스를 여러 곳에서 사용
3. **테스트 용이**: 서비스 로직만 독립적으로 테스트

### TrendTracker의 서비스들

| 서비스 | 역할 |
|--------|------|
| `search_service.py` | Tavily API로 뉴스 검색 |
| `ai_service.py` | Gemini API로 뉴스 요약 |

In [14]:
# ============================================
# 🔍 서비스 레이어 패턴 이해하기
# ============================================
# 서비스 레이어가 어떻게 동작하는지 간단한 예시로 이해해봅시다.

from dataclasses import dataclass
from typing import List

# 도메인 모델
@dataclass
class WeatherInfo:
    city: str
    temperature: float
    condition: str

# 서비스 레이어 (외부 API와 통신 담당)
class WeatherService:
    """날씨 정보를 가져오는 서비스"""
    
    def __init__(self, api_key: str):
        self.api_key = api_key
        # 실제로는 API 클라이언트 초기화
    
    def get_weather(self, city: str) -> WeatherInfo:
        """
        도시의 날씨를 조회합니다.
        
        실제로는 여기서 API 호출이 일어남!
        """
        # 시뮬레이션 (실제로는 requests.get() 등으로 API 호출)
        fake_data = {
            "서울": WeatherInfo("서울", 5.0, "맑음"),
            "부산": WeatherInfo("부산", 8.0, "흐림"),
        }
        
        if city not in fake_data:
            raise ValueError(f"{city}의 날씨 정보가 없습니다")
        
        return fake_data[city]

# 서비스 사용
service = WeatherService(api_key="my-api-key")

print("=== 서비스 레이어 사용 예시 ===")
try:
    weather = service.get_weather("서울")
    print(f"도시: {weather.city}")
    print(f"기온: {weather.temperature}°C")
    print(f"상태: {weather.condition}")
except ValueError as e:
    print(f"오류: {e}")

# 💡 서비스 레이어는 외부 API 호출을 캡슐화합니다!

=== 서비스 레이어 사용 예시 ===
도시: 서울
기온: 5.0°C
상태: 맑음


---

# ✏️ 가이드 실습: TODO 따라하기

---

## ✏️ 실습 1: 기본 예외 처리하기

배운 내용을 직접 코딩해봅시다!

**목표**: try-except로 안전하게 나눗셈 함수 만들기

**힌트**:
- `try:` 블록 안에 위험한 코드
- `except ZeroDivisionError:` 로 0 나눗셈 처리

In [ ]:
# ============================================
# ✏️ 실습 1: 기본 예외 처리하기
# ============================================
# 목표: 안전한 나눗셈 함수 만들기

from typing import Optional

# TODO 1: safe_divide 함수를 작성하세요
# - 매개변수: a (int), b (int)
# - 반환값: Optional[float] (성공 시 결과, 실패 시 None)
# - try-except로 ZeroDivisionError 처리
# 힌트: def 함수명(a: int, b: int) -> Optional[float]:


# >>> 정답 (학습 후 주석 해제하여 확인) <<<
# def safe_divide(a: int, b: int) -> Optional[float]:
#     try:
#         return a / b
#     except ZeroDivisionError:
#         print("❌ 0으로 나눌 수 없습니다")
#         return None

# 테스트
# print(f"10 / 2 = {safe_divide(10, 2)}")  # 5.0
# print(f"10 / 0 = {safe_divide(10, 0)}")  # None

### 💡 실습 1 해설

**TODO 1 해설**: `try` 블록 안에 나눗셈을 넣고, `except ZeroDivisionError`로 0 나눗셈 오류를 잡습니다. 오류 시 `None`을 반환하여 호출자가 실패를 알 수 있게 합니다.

✅ 잘 따라오셨나요? 다음 실습으로 넘어갑시다!

## ✏️ 실습 2: 커스텀 예외 클래스 만들기

프로젝트에서 사용하는 것처럼 커스텀 예외를 만들어봅시다!

**목표**: AppError 클래스 만들고 사용하기

**힌트**:
- `class 예외명(Exception):` 으로 정의
- `raise 예외명(...)` 으로 발생

In [ ]:
# ============================================
# ✏️ 실습 2: 커스텀 예외 클래스 만들기
# ============================================
# 목표: AppError 커스텀 예외 만들고 사용하기

# TODO 1: AppError 예외 클래스를 정의하세요
# - Exception을 상속
# - __init__에서 error_type(str)을 받아 self.error_type에 저장
# 힌트: class AppError(Exception):


# >>> 정답 (학습 후 주석 해제하여 확인) <<<
# class AppError(Exception):
#     def __init__(self, error_type: str):
#         self.error_type = error_type
#         super().__init__(error_type)

# TODO 2: 에러 메시지 딕셔너리를 만드세요
# - "invalid_input": "입력값이 올바르지 않습니다"
# - "not_found": "데이터를 찾을 수 없습니다"


# >>> 정답 (학습 후 주석 해제하여 확인) <<<
# ERROR_MESSAGES = {
#     "invalid_input": "입력값이 올바르지 않습니다",
#     "not_found": "데이터를 찾을 수 없습니다"
# }

# TODO 3: validate_age 함수를 만드세요
# - 나이가 0 미만이면 AppError("invalid_input") 발생
# - 정상이면 나이 반환


# >>> 정답 (학습 후 주석 해제하여 확인) <<<
# def validate_age(age: int) -> int:
#     if age < 0:
#         raise AppError("invalid_input")
#     return age

# 테스트
# try:
#     result = validate_age(-5)
#     print(f"나이: {result}")
# except AppError as e:
#     message = ERROR_MESSAGES.get(e.error_type, "알 수 없는 오류")
#     print(f"❌ {message}")

### 💡 실습 2 해설

**TODO 1 해설**: `Exception`을 상속하여 커스텀 예외를 만듭니다. `error_type`으로 어떤 종류의 오류인지 구분합니다.

**TODO 2 해설**: 에러 타입을 사용자 친화적 메시지로 변환하는 딕셔너리입니다.

**TODO 3 해설**: `raise`로 예외를 발생시킵니다. 호출자가 `try-except`로 처리합니다.

✅ 잘 따라오셨나요? 다음 실습으로 넘어갑시다!

## ✏️ 실습 3: 간단한 서비스 함수 만들기

프로젝트의 서비스 레이어처럼 함수를 만들어봅시다!

**목표**: API 호출을 시뮬레이션하는 서비스 함수 만들기

**힌트**:
- 함수 내에서 예외 처리
- 성공 시 데이터 반환, 실패 시 커스텀 예외

In [ ]:
# ============================================
# ✏️ 실습 3: 간단한 서비스 함수 만들기
# ============================================
# 목표: 사용자 정보를 조회하는 서비스 함수 만들기

from dataclasses import dataclass
from typing import Optional

# 도메인 모델
@dataclass
class User:
    id: int
    name: str
    email: str

# 가짜 데이터베이스
FAKE_DB = {
    1: User(1, "홍길동", "hong@email.com"),
    2: User(2, "김철수", "kim@email.com"),
}

# 커스텀 예외 (위에서 정의한 것 사용)
class AppError(Exception):
    def __init__(self, error_type: str):
        self.error_type = error_type
        super().__init__(error_type)

# TODO 1: get_user_by_id 서비스 함수를 만드세요
# - user_id를 받아 User 객체를 반환
# - 사용자가 없으면 AppError("not_found") 발생
# 힌트: FAKE_DB에서 찾기, 없으면 raise


# >>> 정답 (학습 후 주석 해제하여 확인) <<<
# def get_user_by_id(user_id: int) -> User:
#     """사용자 ID로 사용자 정보를 조회합니다."""
#     user = FAKE_DB.get(user_id)
#     if user is None:
#         raise AppError("not_found")
#     return user

# TODO 2: 함수를 사용하여 사용자 조회하기
# - ID 1번 조회 (성공)
# - ID 999번 조회 (실패 → 예외 처리)


# >>> 정답 (학습 후 주석 해제하여 확인) <<<
# print("=== 사용자 조회 테스트 ===")

# # 성공 케이스
# try:
#     user = get_user_by_id(1)
#     print(f"✅ 조회 성공: {user.name} ({user.email})")
# except AppError as e:
#     print(f"❌ 조회 실패: {e.error_type}")

# # 실패 케이스
# try:
#     user = get_user_by_id(999)
#     print(f"✅ 조회 성공: {user.name}")
# except AppError as e:
#     print(f"❌ 조회 실패: {e.error_type}")

=== 사용자 조회 테스트 ===
✅ 조회 성공: 홍길동 (hong@email.com)
❌ 조회 실패: not_found


### 💡 실습 3 해설

**TODO 1 해설**: 서비스 함수는 데이터를 조회하고, 없으면 적절한 예외를 발생시킵니다. 이렇게 하면 호출자가 오류 상황을 명확히 알 수 있습니다.

**TODO 2 해설**: 서비스 함수를 사용할 때는 항상 `try-except`로 감싸서 예외를 처리합니다.

✅ 서비스 레이어 패턴을 이해하셨나요? 이제 실제 프로젝트 코드를 분석해봅시다!

---

# 🔎 실제 프로젝트에서는 어떻게 쓰일까요?

---

## 🔎 utils/exceptions.py 분석

**파일**: `utils/exceptions.py`
**역할**: 애플리케이션 전용 커스텀 예외 클래스

In [ ]:
# ============================================
# 🔎 initial_version의 utils/exceptions.py 코드
# ============================================
# 이 코드는 애플리케이션 전용 예외 클래스입니다.

class AppError(Exception):
    """애플리케이션 전용 커스텀 예외 클래스"""
    
    def __init__(self, error_type: str):
        self.error_type = error_type  # 에러 종류 (예: "api_key_invalid")
        super().__init__(error_type)  # 부모 클래스 초기화

# 💡 이 클래스가 하는 역할:
# - 모든 API 관련 오류를 하나의 타입으로 통일
# - error_type으로 어떤 종류의 오류인지 구분
# - UI에서 error_type을 사용자 친화적 메시지로 변환

# 사용 예시
print("=== AppError 사용 예시 ===")

# 예외 발생 및 처리
try:
    raise AppError("rate_limit_exceeded")
except AppError as e:
    print(f"에러 타입: {e.error_type}")
    # UI에서는 이를 "요청이 너무 많습니다..."로 변환

## 🔎 services/search_service.py 분석

**파일**: `services/search_service.py`
**역할**: Tavily API로 뉴스를 검색하는 서비스

In [16]:
# ============================================
# 🔎 initial_version의 services/search_service.py 코드 (학습용 정리)
# ============================================
# 이 코드는 Tavily API로 뉴스를 검색하는 서비스입니다.

# 실제 코드의 구조를 설명합니다 (실행 불가 - 설명용)

code_explanation = """
from typing import List
from tavily import TavilyClient          # Tavily API 클라이언트
from config.settings import Settings      # 환경 설정
from domain.news_article import NewsArticle  # 도메인 모델
from utils.exceptions import AppError     # 커스텀 예외

def search_news(keyword: str, num_results: int = 5) -> List[NewsArticle]:
    \"\"\"
    Tavily API를 사용하여 뉴스 기사를 검색합니다.
    
    Args:
        keyword: 검색할 키워드
        num_results: 가져올 결과 개수 (기본값: 5)
    Returns:
        List[NewsArticle]: 검색된 뉴스 기사 리스트
    Raises:
        AppError: API 오류 발생 시
    \"\"\"
    settings = Settings()  # 환경 설정 로드
    
    try:
        # 1. API 클라이언트 초기화
        client = TavilyClient(api_key=settings.TAVILY_API_KEY)
        
        # 2. 충분한 결과를 가져오기 (최신순 정렬을 위해)
        fetch_count = max(num_results * 3, 20)
        
        # 3. API 호출
        response = client.search(
            query=keyword,
            search_depth="advanced",           # 고급 검색 (날짜 정보 포함)
            include_domains=settings.SEARCH_DOMAINS,  # 특정 도메인만
            max_results=fetch_count,
            topic="news"                       # 뉴스 모드
        )
        
        # 4. 최신순 정렬
        results = response.get('results', [])
        results.sort(key=lambda x: x.get('published_date', ''), reverse=True)
        results = results[:num_results]  # 요청된 개수만큼
        
        # 5. NewsArticle 리스트로 변환
        articles = []
        for res in results:
            articles.append(NewsArticle(
                title=res.get('title', '제목 없음'),
                url=res.get('url', ''),
                snippet=res.get('content', ''),
                pub_date=res.get('published_date')
            ))
        return articles
        
    except Exception as e:
        # 6. 예외를 AppError로 변환
        error_str = str(e).lower()
        if "401" in error_str or "api key" in error_str:
            raise AppError("api_key_invalid")
        elif "429" in error_str:
            raise AppError("rate_limit_exceeded")
        else:
            raise AppError("network_error")
"""

print("=== search_service.py 코드 구조 ===")
print(code_explanation)

print("\n💡 핵심 포인트:")
print("1. Settings에서 API 키를 가져옴")
print("2. TavilyClient로 API 호출")
print("3. 결과를 최신순 정렬")
print("4. NewsArticle 리스트로 변환")
print("5. 모든 예외를 AppError로 통일")

=== search_service.py 코드 구조 ===

from typing import List
from tavily import TavilyClient          # Tavily API 클라이언트
from config.settings import Settings      # 환경 설정
from domain.news_article import NewsArticle  # 도메인 모델
from utils.exceptions import AppError     # 커스텀 예외

def search_news(keyword: str, num_results: int = 5) -> List[NewsArticle]:
    """
    Tavily API를 사용하여 뉴스 기사를 검색합니다.

    Args:
        keyword: 검색할 키워드
        num_results: 가져올 결과 개수 (기본값: 5)
    Returns:
        List[NewsArticle]: 검색된 뉴스 기사 리스트
    Raises:
        AppError: API 오류 발생 시
    """
    settings = Settings()  # 환경 설정 로드

    try:
        # 1. API 클라이언트 초기화
        client = TavilyClient(api_key=settings.TAVILY_API_KEY)

        # 2. 충분한 결과를 가져오기 (최신순 정렬을 위해)
        fetch_count = max(num_results * 3, 20)

        # 3. API 호출
        response = client.search(
            query=keyword,
            search_depth="advanced",           # 고급 검색 (날짜 정보 포함)
            include_domains=settings.SEARCH_DOMAINS,  # 특

## 🔎 services/ai_service.py 분석

**파일**: `services/ai_service.py`
**역할**: Gemini API로 뉴스를 요약하는 서비스

In [17]:
# ============================================
# 🔎 initial_version의 services/ai_service.py 코드 (학습용 정리)
# ============================================
# 이 코드는 Gemini API로 뉴스를 요약하는 서비스입니다.

code_explanation = """
from typing import List
from google import genai               # Gemini API 클라이언트
from config.settings import Settings
from domain.news_article import NewsArticle
from utils.exceptions import AppError

def summarize_news(articles: List[NewsArticle]) -> str:
    \"\"\"
    Google Gemini API를 사용하여 뉴스 기사들을 요약합니다.
    
    Args:
        articles: 요약할 뉴스 기사 리스트
    Returns:
        str: 한국어 요약 결과
    Raises:
        AppError: API 오류 발생 시
    \"\"\"
    if not articles:
        return "요약할 기사가 없습니다."
    
    settings = Settings()
    
    try:
        # 1. API 클라이언트 초기화
        client = genai.Client(api_key=settings.GEMINI_API_KEY)
        
        # 2. 프롬프트 구성
        news_context = ""
        for i, article in enumerate(articles, 1):
            news_context += f"{i}. 제목: {article.title}\n"
            news_context += f"   내용: {article.snippet}\n\n"
        
        prompt = f\"\"\"
다음 뉴스 기사들의 핵심 내용을 한국어로 요약해주세요:
- 불릿 포인트 형식으로 최대 5개 항목
- 각 항목은 1~2문장

[뉴스 목록]
{news_context}
\"\"\"
        
        # 3. API 호출
        response = client.models.generate_content(
            model=settings.GEMINI_MODEL,
            contents=prompt
        )
        
        return response.text or "요약 결과를 생성하지 못했습니다."
        
    except Exception as e:
        # 4. 예외를 AppError로 변환
        error_str = str(e).lower()
        if "401" in error_str or "api key" in error_str:
            raise AppError("api_key_invalid")
        elif "429" in error_str:
            raise AppError("rate_limit_exceeded")
        else:
            raise AppError("ai_error")
"""

print("=== ai_service.py 코드 구조 ===")
print(code_explanation)

print("\n💡 핵심 포인트:")
print("1. 기사 리스트를 프롬프트로 변환")
print("2. 한국어 요약을 요청하는 프롬프트 사용")
print("3. Gemini API 호출")
print("4. 결과 텍스트 반환")
print("5. 모든 예외를 AppError로 통일")

=== ai_service.py 코드 구조 ===

from typing import List
from google import genai               # Gemini API 클라이언트
from config.settings import Settings
from domain.news_article import NewsArticle
from utils.exceptions import AppError

def summarize_news(articles: List[NewsArticle]) -> str:
    """
    Google Gemini API를 사용하여 뉴스 기사들을 요약합니다.

    Args:
        articles: 요약할 뉴스 기사 리스트
    Returns:
        str: 한국어 요약 결과
    Raises:
        AppError: API 오류 발생 시
    """
    if not articles:
        return "요약할 기사가 없습니다."

    settings = Settings()

    try:
        # 1. API 클라이언트 초기화
        client = genai.Client(api_key=settings.GEMINI_API_KEY)

        # 2. 프롬프트 구성
        news_context = ""
        for i, article in enumerate(articles, 1):
            news_context += f"{i}. 제목: {article.title}
"
            news_context += f"   내용: {article.snippet}

"

        prompt = f"""
다음 뉴스 기사들의 핵심 내용을 한국어로 요약해주세요:
- 불릿 포인트 형식으로 최대 5개 항목
- 각 항목은 1~2문장

[뉴스 목록]
{news_context}
"""

        # 3. API 호출
 

## ✏️ 프로젝트 코드 따라해보기

In [18]:
# ============================================
# ✏️ 프로젝트 코드 따라해보기: 서비스 함수 만들기
# ============================================
# 프로젝트의 서비스 패턴을 따라 간단한 버전을 만들어봅시다.

from dataclasses import dataclass
from typing import List, Optional

# 커스텀 예외
class AppError(Exception):
    def __init__(self, error_type: str):
        self.error_type = error_type
        super().__init__(error_type)

# 도메인 모델
@dataclass
class NewsArticle:
    title: str
    url: str
    snippet: str
    pub_date: Optional[str] = None

# 가짜 API 응답 (실제로는 API에서 옴)
FAKE_API_RESPONSE = {
    "results": [
        {"title": "AI 기술 발전", "url": "https://1.com", "content": "AI가 빠르게 발전하고 있습니다.", "published_date": "2024-01-20"},
        {"title": "로봇 시대 도래", "url": "https://2.com", "content": "로봇이 일상에 들어옵니다.", "published_date": "2024-01-19"},
        {"title": "미래 기술 전망", "url": "https://3.com", "content": "2024년 기술 트렌드.", "published_date": None},
    ]
}

# TODO 1: search_news 함수를 만드세요 (간소화 버전)
# - keyword를 받아 List[NewsArticle]을 반환
# - FAKE_API_RESPONSE에서 데이터를 가져옴
# - published_date 기준 내림차순 정렬
# - NewsArticle 리스트로 변환하여 반환


# >>> 정답 <<<
def search_news(keyword: str, num_results: int = 5) -> List[NewsArticle]:
    """뉴스를 검색합니다 (시뮬레이션)"""
    try:
        results = FAKE_API_RESPONSE.get('results', [])
        
        # 최신순 정렬 (날짜 없는 것은 뒤로)
        results.sort(
            key=lambda x: x.get('published_date') or '',
            reverse=True
        )
        results = results[:num_results]
        
        # NewsArticle로 변환
        articles = []
        for res in results:
            articles.append(NewsArticle(
                title=res.get('title', '제목 없음'),
                url=res.get('url', ''),
                snippet=res.get('content', ''),
                pub_date=res.get('published_date')
            ))
        return articles
        
    except Exception as e:
        raise AppError("network_error")

# 테스트
articles = search_news("AI")
print("=== 검색 결과 ===")
for i, article in enumerate(articles, 1):
    print(f"{i}. {article.title} ({article.pub_date})")

=== 검색 결과 ===
1. AI 기술 발전 (2024-01-20)
2. 로봇 시대 도래 (2024-01-19)
3. 미래 기술 전망 (None)


---

# 🏆 강사와 함께하는 챌린지

지금까지 배운 내용을 종합하여 문제를 풀어봅시다!

⚠️ **주의**: 이 섹션에는 정답이 없습니다.
- 먼저 스스로 시도해보세요
- 막히면 강사에게 질문하세요
- 함께 풀어보며 이해도를 높입니다

**챌린지 목표**: 완전한 서비스 레이어 구현하기

---

In [ ]:
# ============================================
# 🏆 챌린지: 날씨 서비스 만들기
# ============================================
# 목표: API 호출을 시뮬레이션하는 완전한 날씨 서비스 만들기
#
# ⚠️ 정답이 제공되지 않습니다. 강사와 함께 풀어보세요!

from dataclasses import dataclass
from typing import List, Optional

# TODO 1: WeatherError 커스텀 예외를 만드세요
# - error_type 속성을 가짐
# 힌트: AppError와 동일한 구조

# TODO 2: WeatherInfo 도메인 모델을 만드세요
# - city: str (도시명)
# - temperature: float (기온)
# - humidity: int (습도)
# - condition: str (날씨 상태, 기본값 "맑음")


# TODO 3: get_weather 서비스 함수를 만드세요
# - city를 받아 WeatherInfo를 반환
# - 아래 FAKE_WEATHER_DATA에서 데이터를 가져옴
# - 도시가 없으면 WeatherError("city_not_found") 발생
# - 도시가 빈 문자열이면 WeatherError("invalid_input") 발생

# 테스트용 샘플 데이터
FAKE_WEATHER_DATA = {
    "서울": {"temperature": 5.0, "humidity": 60, "condition": "맑음"},
    "부산": {"temperature": 8.0, "humidity": 70, "condition": "흐림"},
    "제주": {"temperature": 10.0, "humidity": 80, "condition": "비"},
}

# TODO 4: 서비스 함수를 테스트하세요
# - "서울" 조회 (성공)
# - "도쿄" 조회 (실패 - city_not_found)
# - "" 조회 (실패 - invalid_input)
# - 각각 try-except로 처리하고 결과 출력




In [ ]:
# ============================================
# 🏆 챌린지: 요약 서비스 만들기
# ============================================
# 목표: 뉴스 요약 서비스 (AI 없이 간단 버전) 만들기
#
# ⚠️ 정답이 제공되지 않습니다. 강사와 함께 풀어보세요!

from dataclasses import dataclass
from typing import List, Optional

# 도메인 모델 (독립 실행을 위해 여기에 정의)
@dataclass
class NewsArticle:
    title: str
    url: str
    snippet: str
    pub_date: Optional[str] = None


# TODO 1: summarize_articles 함수를 만드세요
# - List[NewsArticle]을 받아 요약 문자열을 반환
# - 각 기사의 제목을 번호와 함께 나열
# - 빈 리스트면 "요약할 기사가 없습니다." 반환
# 
# 예시 출력:
# "[뉴스 요약]\n1. AI 기술 발전\n2. 로봇 시대 도래\n..."

# TODO 2: 테스트
# - 기사 3개로 테스트
# - 빈 리스트로 테스트

# 테스트용 샘플 데이터
sample_articles = [
    NewsArticle("AI 기술 발전", "https://1.com", "AI가 빠르게 발전하고 있습니다.", "2024-01-20"),
    NewsArticle("로봇 시대 도래", "https://2.com", "로봇이 일상에 들어옵니다.", "2024-01-19"),
    NewsArticle("미래 기술 전망", "https://3.com", "2024년 기술 트렌드.", None),
]



'[뉴스 요약]\n2. AI 기술 발전 \n3. 로봇 시대 도래 \n4. 미래 기술 전망 \n'

In [ ]:
# ============================================
# 🏆 보너스 챌린지: 전체 파이프라인 구현
# ============================================
# 시간이 남으면 도전해보세요!
#
# ⚠️ 정답이 제공되지 않습니다.

from dataclasses import dataclass
from typing import List, Optional, Dict, Any

# 도메인 모델 (독립 실행을 위해 여기에 정의)
@dataclass
class NewsArticle:
    title: str
    url: str
    snippet: str
    pub_date: Optional[str] = None


# 커스텀 예외 (독립 실행을 위해 여기에 정의)
class AppError(Exception):
    def __init__(self, error_type: str):
        self.error_type = error_type
        super().__init__(error_type)


# 가짜 API 응답 (실제로는 API에서 옴)
FAKE_API_RESPONSE = {
    "results": [
        {"title": "AI 기술 발전", "url": "https://1.com", "content": "AI가 빠르게 발전하고 있습니다.", "published_date": "2024-01-20"},
        {"title": "로봇 시대 도래", "url": "https://2.com", "content": "로봇이 일상에 들어옵니다.", "published_date": "2024-01-19"},
        {"title": "미래 기술 전망", "url": "https://3.com", "content": "2024년 기술 트렌드.", "published_date": None},
    ]
}


# TODO: 검색 → 요약 전체 파이프라인 구현
#
# 요구사항:
# 1. search_news(keyword: str) 함수 만들기 - FAKE_API_RESPONSE에서 데이터 가져오기
# 2. summarize_articles(articles: List[NewsArticle]) 함수 만들기 - 기사 제목 나열
# 3. search_and_summarize(keyword: str) 함수 만들기
#    - 내부에서 search_news() 호출
#    - 결과를 summarize_articles()로 요약
#    - 검색 결과와 요약을 모두 담은 결과 반환
# 4. 모든 오류를 적절히 처리

# 결과 형식 예시:
# {
#     "keyword": "AI",
#     "articles": [...],
#     "summary": "[뉴스 요약]...",
#     "success": True
# }

def search_news(keyword: str) -> List[NewsArticle]:
    if not keyword:
        raise AppError("invalid_input")
    
    news = FAKE_API_RESPONSE.get('results', [])
    result = []
    for article in news:
        result.append(NewsArticle(
            title=article.get('title', '제목 없음'),
            url=article.get('url', ''),
            snippet=article.get('content', ''),
            pub_date=article.get('published_date')
        ))
    return result

def summarize_articles(articles: List[NewsArticle]) -> str:
    if not articles:
        return "요약할 기사가 없습니다."
    
    result = "[뉴스 요약]\n"
    for i, article in enumerate(articles, 1):
        result += f"{i}. {article.title}\n"
    return result

def search_and_summarize(keyword: str) -> dict:
    try:
        articles = search_news(keyword)
        return {
            "keyword": keyword,
            "articles": articles,
            "summary": summarize_articles(articles),
            "success": True,
        }
    except AppError:
        return {
            "keyword": keyword,
            "articles": [],
            "summary": "",
            "success": False,
        }


---

# 📋 정리

## 이번 Phase에서 배운 것

| 개념 | 한 줄 요약 |
|------|-----------|
| **API** | 프로그램 간 소통 방법 (웨이터) |
| **HTTP 요청/응답** | 웹에서 데이터를 주고받는 방법 |
| **상태 코드** | 요청 결과를 나타내는 숫자 (200=성공) |
| **try-except** | 오류를 안전하게 처리하는 방법 (안전벨트) |
| **커스텀 예외** | 우리만의 예외 클래스 (AppError) |
| **서비스 레이어** | API 호출을 담당하는 계층 (영업부) |

## 핵심 코드 패턴

```python
# 서비스 함수 기본 패턴
from utils.exceptions import AppError

def service_function(param):
    try:
        # 1. API 클라이언트 초기화
        client = APIClient(api_key=settings.API_KEY)
        
        # 2. API 호출
        response = client.call(param)
        
        # 3. 결과 변환 및 반환
        return convert_to_domain_model(response)
        
    except Exception as e:
        # 4. 예외를 AppError로 변환
        if "401" in str(e):
            raise AppError("api_key_invalid")
        raise AppError("network_error")
```

## 다음 Phase에서는?

- **Phase 4**: 리포지토리 레이어 - 데이터 관리
  - CSV 파일로 데이터 저장/읽기
  - Repository 패턴 이해하기
  - pandas DataFrame 활용하기
- 이번에 배운 서비스 결과를 저장하는 방법을 배웁니다!

## ✅ 학습 체크리스트

- [ ] API가 무엇인지 설명할 수 있다 (식당 비유)
- [ ] HTTP 상태 코드의 의미를 안다 (200, 401, 429, 500)
- [ ] try-except로 예외를 처리할 수 있다
- [ ] 커스텀 예외 클래스를 만들 수 있다
- [ ] search_service.py, ai_service.py의 구조를 이해했다
- [ ] 챌린지를 강사와 함께 풀어봤다

---

🎉 **Phase 3 학습을 완료하셨습니다!**